In [1]:
from LibPath import *
exec(runBasic())

# <font face='comic sans ms' color ='red'>Working Area</font>

## <font face='comic sans ms' color ='DarkRed'>Sunday, 08.05.2022</font>

In [2]:
import pyautogui as pg
import cv2
import time

In [3]:
class Board:
    def __init__(self):
        self.RowCol=(10,6)
        self.TopLeftCorner=(1266,114)
        self.BottomRightCorner=(1832,1060)
        self.matrix=[[0 for i in range(6)] for j in range(10)]
    def setRowCol(self,rc:tuple):
        self.RowCol=rc
    def setTopLeftCorner(self,tlc:tuple):
        self.TopLeftCorner=tlc
    def setBottomRightCorner(self,brc:tuple):
        self.BottomRightCorner=brc

    def _getSteps(self):
        x1,y1=self.TopLeftCorner
        x2,y2=self.BottomRightCorner
        r,c=self.RowCol
        xStep=(x2-x1)//c
        yStep=(y2-y1)//r
        return xStep,yStep

    def takeSS(self):
        x1,y1=self.TopLeftCorner
        x2,y2=self.BottomRightCorner
        width=x2-x1
        height=y2-y1
        pg.screenshot('ss.png',region=(*self.TopLeftCorner, width, height))

    def execute(self):
        xStep,yStep=self._getSteps()
        start=(self.TopLeftCorner[0]+xStep//2,self.TopLeftCorner[1]+yStep//2)
        pg.click(start[0],start[1],duration=.5)

    def getArray(self,img):
        matrix=[]
        x,y=self.TopLeftCorner
        xStep,yStep=self._getSteps()
        for i in range(self.RowCol[0]):
            col=[]
            for j in range(self.RowCol[1]):
                image=img[xStep*i:xStep*(i+1),yStep*j:yStep*(j+1)]
                col.append(self.getNumber(image))
            matrix.append(col)
        return matrix

    def getNumber(self,image):
        for i in range(0,7):
            errorL2 = cv2.norm( image, cv2.imread(f'{i}.jpg',1), cv2.NORM_L2 )
            similarity = 1 - errorL2 / ( image.shape[0] * image.shape[1] )
            print(i,similarity)
        print()
        return 0

In [4]:
b=Board()
# b.execute()
# b.takeSS()


In [6]:
im=cv2.imread('ss1.png',1)
# image=im[0:93,0:94]
# # image=im[93:186,0:94]
# cv2.imshow('sdjn',im)
# cv2.imwrite('img\\filename.jpg',im)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [7]:
im.shape

(946, 566, 3)

In [8]:
mat=b.getArray(im)
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\core\src\norm.cpp:671: error: (-215:Assertion failed) mask.empty() || mask.type() == CV_8U in function 'cv::norm'


In [4]:
def Number(roi):
    for i in range(1,10):
        template=cv2.imread('s{}.jpg'.format(i),0)
        gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
        res=cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)
        threshold=0.9
        loc=where(res>=threshold)
        w,h=template.shape[::-1]
        for pt in zip(*loc[::-1]):
            return i
    return 0

In [16]:
b._getSteps()

(93, 94)

## <font face='comic sans ms' color ='DarkTurquoise'>Monday, 09.05.2022</font>

### Old code

In [ ]:
import cv2
from numpy import *
        
def fillNum(given,thick,expt):
    f1=(p2[0]-p1[0])//9
    f2=(p2[1]-p1[1])//9
    for i in given:
        c=0
        for j in expt:
            if i==j:
                c=1
        if c==0:
            cv2.putText(img1,'{}'.format(data[i[0]][i[1]]),(35+f1*(i[1]),70+f2*(i[0])),cv2.FONT_HERSHEY_SCRIPT_COMPLEX,1.5,(255,0,0),thick)

def Number(roi):
    for i in range(1,10):
        template=cv2.imread('s{}.jpg'.format(i),0)
        gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
        res=cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)
        threshold=0.9
        loc=where(res>=threshold)
        w,h=template.shape[::-1]
        for pt in zip(*loc[::-1]):
            return i
    return 0

def clearImage():
    _,th=cv2.threshold(img,120,255,cv2.THRESH_BINARY)
    kernal=ones([3,3],uint8)
    erosion=cv2.erode(th,kernal)
    return erosion

def detect(temp):
    corners=cv2.goodFeaturesToTrack(temp,150,0.01,10)
    corners =int0(corners)
    points=[]
    for i in corners:
        x,y=i.ravel()
        points.append((x,y))
    return min(points),max(points)

def getPoints(erosion):
    temp=zeros([img.shape[0],img.shape[1]],uint8)
    temp[:]=255
    edge=cv2.Canny(erosion,108,22,apertureSize=3)
    lines=cv2.HoughLines(edge,1,pi/180,200)
    for line in lines:
        rho,theta=line[0]
        a=cos(theta)
        b=sin(theta)
        x0=a*rho
        y0=b*rho
        x1=int(x0+1000*(-b))
        y1=int(y0+1000*a)
        x2=int(x0-1000*(-b))
        y2=int(y0-1000*a)
        cv2.line(temp,(x1,y1),(x2,y2),0,2)
    return detect(temp)
    
def getArray(img,p1,p2):
    f1=(p2[0]-p1[0])//9
    f2=(p2[1]-p1[1])//9
    global data
    for i in range(9):
        for j in range(9):
            image=img[(p1[0]+f1*i):(p1[0]+f1*(i+1)), (p1[1]+f2*j):(p1[1]+f2*(j+1))]
            data[i][j]=Number(image)

def rules(value,x,y):
    for i in range(9):
        if value==data[x][i] or value==data[i][y]:
            return False
    r1=(x//3)*3
    r2=(y//3)*3
    for i in range(r1,r1+3):
        for j in range(r2,r2+3):
            if value==data[i][j]:
                return False
    return True
    
def filled():
    given=[]
    for i in range(9):
        for j in range(9):
            if data[i][j]!=0:
                given.append((i,j))
    return given

def isVacant(a):
    given=filled()
    for i in given:
        if a==i:
            return False
    return True

def UniqueNumber(data):
    for i in range(9):
        for j in range(9):
            if isVacant((i,j)):
                possibility=0
                possValue=0
                for value in range(1,10):
                    if rules(value,i,j):
                        possibility+=1
                        possValue=value
                if possibility==1:
                    data[i][j]=possValue

def EliminatingBoxes(data):
    for row in range(0,9,3):
        for column in range(0,9,3):
            for value in range(1,10):
                position=None
                positionValue=None
                occurence=0
                for i in range(row,row+3):
                    for j in range(column,column+3):
                        if isVacant((i,j)) and rules(value,i,j):
                            position=(i,j)
                            occurence+=1
                            positionValue=value
                if occurence==1:
                    data[position[0]][position[1]]=positionValue
                                        
def solve(data):
    while True:
        temp=len(filled())
        UniqueNumber(data)
        EliminatingBoxes(data)
        if temp-len(filled())==0:
            break

            
img=cv2.imread('ImgSud2.jpg',1)
cv2.imshow('img',img)
erosion=clearImage()
p1,p2=getPoints(erosion)

img1=img

data=array([[0 for x in range(9)] for y in range(9)])
getArray(img,p1,p2)

given=filled()

solve(data)  

fillNum(filled(),1,given)
cv2.imshow('Answer',img1)

cv2.waitKey(0)
cv2.destroyAllWindows()

### New Code

In [9]:
def Number(roi):
    for i in range(1,10):
        template=cv2.imread('s{}.jpg'.format(i),0)
        gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
        res=cv2.matchTemplate(gray,template,cv2.TM_CCOEFF_NORMED)
        threshold=0.9
        loc=where(res>=threshold)
        w,h=template.shape[::-1]
        for pt in zip(*loc[::-1]):
            return i
    return 0

def getArray(img,p1,p2):
    f1=(p2[0]-p1[0])//9
    f2=(p2[1]-p1[1])//9
    global data
    for i in range(9):
        for j in range(9):
            image=img[(p1[0]+f1*i):(p1[0]+f1*(i+1)), (p1[1]+f2*j):(p1[1]+f2*(j+1))]
            data[i][j]=Number(image)

In [ ]:
getArray(img)

## <font face='comic sans ms' color ='HotPink'>Tuesday, 10.05.2022</font>

In [5]:
class MatchPics:
    def setImage1(self,img):
        self.img1=img
    def setImage2(self,img):
        self.img2=img
    def match(self):
        img_gray = cv2.cvtColor(self.img1, cv2.COLOR_BGR2GRAY)
        template = cv2.cvtColor(self.img2, cv2.COLOR_BGR2GRAY)
        # w, h = template.shape[::-1]
        res = cv2.matchTemplate(img_gray,template,cv2.TM_CCOEFF_NORMED)
        threshold = 0.8
        loc = np.where( res >= threshold)
        for pt in zip(*loc[::-1]):
            return 1
        return 0

In [26]:
img1=cv2.imread('ss1.png')
img2=cv2.imread('2.jpg')

In [24]:
m=MatchPics()
m.setImage1(img1)
m.setImage2(img2)
m.match()

0

In [13]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
img_rgb = cv.imread('ss2.png')
img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
template = cv.imread('0.png',0)
w, h = template.shape[::-1]
res = cv.matchTemplate(img_gray,template,cv.TM_CCOEFF_NORMED)
threshold = 0.8
loc = np.where( res >= threshold)
for pt in zip(*loc[::-1]):
    # print('True')
    cv.rectangle(img_rgb, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)
    cv.rectangle(template, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

cv2.imshow('img_rgb',img_rgb)
cv2.imshow('template',template)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [27]:
from Minesweeper import MatchPics
m=MatchPics()
m.setImage1(img1)
m.setImage2(img2)
m.match()

1

## <font face='comic sans ms' color ='DarkBlue'>Wednesday, 11.05.2022</font>

In [2]:
# exit()
quit()
print(5)

5


: 

In [8]:
import cv2
import Minesweeper as ms
b=ms.Board()
b.extractArray(cv2.imread('ss.png'))
# b.takeSS()

[[0, 0, 0, 2, 9, 9],
 [0, 0, 0, 2, 9, 9],
 [0, 0, 0, 3, 1, 9],
 [0, 0, 0, 0, 1, 9],
 [0, 2, 2, 2, 1, 9],
 [1, 1, 9, 9, 9, 9],
 [9, 9, 1, 1, 1, 9],
 [9, 9, 1, 0, 2, 1],
 [9, 9, 1, 3, 0, 0],
 [9, 9, 9, 2, 0, 0]]

In [3]:
b._getSteps()

(94, 94)

In [4]:
img1=cv2.imread('img/(2,0).png')
img2=cv2.imread('ss.png')
cv2.imshow('img1',img1)
cv2.imshow('img2',img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [1]:
a=[[9, 2, 6, 6, 6, 2],
 [1, 3, 6, 5, 6, 2],
 [1, 6, 2, 2, 1, 1],
 [1, 1, 1, 9, 9, 9],
 [9, 9, 9, 9, 9, 9],
 [1, 1, 2, 2, 3, 2],
 [3, 6, 3, 6, 6, 6],
 [6, 6, 4, 3, 5, 0],
 [0, 6, 0, 0, 6, 6],
 [6, 0, 0, 0, 0, 0]]

In [7]:
m=ms.MatchPics()
m.setImage1(img2)
m.setImage2(img1)
m.match()

1

In [20]:
a[3][4]=5
a

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 5, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

In [13]:
r,c=1,1
[(r-1,c-1),(r-1,c),(r-1,c+1),(r,c-1),(r,c+1),(r+1,c-1),(r+1,c),(r+1,c+1)]

[(0, 0), (0, 1), (0, 2), (1, 0), (1, 2), (2, 0), (2, 1), (2, 2)]

In [1]:
aug=[[0 for i in range(6)] for j in range(10)]
a=[[0, 0, 0, 2, 9, 9],
 [0, 0, 0, 2, 9, 9],
 [0, 0, 0, 3, 1, 9],
 [0, 0, 0, 0, 1, 9],
 [0, 2, 2, 2, 1, 9],
 [1, 1, 9, 9, 9, 9],
 [9, 9, 1, 1, 1, 9],
 [9, 9, 1, 0, 2, 1],
 [9, 9, 1, 3, 0, 0],
 [9, 9, 9, 2, 0, 0]]

In [2]:
import cv2
import Minesweeper as ms
b=ms.Rule()
b.set_matrix(a)
b.rule_1(aug)

set()


In [3]:
aug

[[0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0]]

In [4]:
a

[[0, 0, 0, 2, 9, 9],
 [0, 0, 0, 2, 9, 9],
 [0, 0, 0, 3, 1, 9],
 [0, 0, 0, 0, 1, 9],
 [0, 2, 2, 2, 1, 9],
 [1, 1, 9, 9, 9, 9],
 [9, 9, 1, 1, 1, 9],
 [9, 9, 1, 0, 2, 1],
 [9, 9, 1, 3, 0, 0],
 [9, 9, 9, 2, 0, 0]]

In [12]:
if {()}:
    print('hi')

hi


In [5]:
{1,2}.union({3,4})

{1, 2, 3, 4}

# <font face='comic sans ms' color ='BurlyWood'>path, file, jupyter</font>

In [6]:
_ = jupyterDB.header(TimeDB.getTimeStamp())

# <font face='comic sans ms' color ='DarkBlue'>Wednesday, 11.05.2022</font>


In [ ]:
Path.openExplorerAt()

In [ ]:
rlib.modb.search("videoDB")

## <font face='comic sans ms' color ='LimeGreen'>pickle</font>

In [30]:
jupyterDB.pickle().listDir()

['jupyterDB.pkl', 'logs.pkl', 'paths.pkl', 'runBasic.pkl']

In [39]:
name = "jupyterDB"
k = jupyterDB.pickle().read(name)

In [45]:
k = jupyterDB.pickle().write(k, name)

In [43]:
# k

In [44]:
k={'createNotebook': [('code', 'from LibPath import *\r\nexec(runBasic())'),
  ('code', ''),
  ('md', "# <font face='comic sans ms' color ='red'>Working Area</font>"),

  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('code', ''),
  ('md',
   "# <font face='comic sans ms' color ='BurlyWood'>path, file, jupyter</font>"),
  ('code', '_ = jupyterDB.header(TimeDB.getTimeStamp())'),
  ('code', 'Path.openExplorerAt()'),
  ('code', 'rlib.modb.search("videoDB")'),
  ('md', "## <font face='comic sans ms' color ='LimeGreen'>pickle</font>"),
  ('code', 'jupyterDB.pickle().listDir()'),
  ('code', 'name = "jupyterDB"\nk = jupyterDB.pickle().read(name)'),
  ('code', 'k = jupyterDB.pickle().write(k, name)'),
  ('code', 'k'),
  ('code','#create a new notebook\njupyterDB.createJupyterNotebook("name")')],
 'fonts': ['purisa', 'comic sans ms', 'ubuntu mono']}